In [3]:
import os, sys
sys.path.append(os.path.join('/home/ring/sumo-svn/', 'tools'))
import traci
import traci.constants as tc
import numpy as np
from sumolib import checkBinary
import datetime
import copy

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import optparse
import pandas as pd
import asyncio
from collections import namedtuple
import random
import matplotlib
import matplotlib.pyplot as plt

In [5]:
class NetworkA1(nn.Module):
    def __init__(self):
        super(NetworkA1, self).__init__()
        # [50, 60, 100, 0, .... , 20, 8]
        self.fc_layer1 = nn.Linear(10, 15) ##CHANGE!! 8
        self.fc_layer2 = nn.Linear(15, 20)
        self.fc_layer3 = nn.Linear(20, 2) ##CHANGE!! 2
    def forward(self, x):
        # x is input
        # Layer 1
        x = F.relu(self.fc_layer1(x))
        # Layer 2
        x = F.relu(self.fc_layer2(x))
        # Layer 3
        x = self.fc_layer3(x)
        return x
model1 = NetworkA1()

In [6]:
class NetworkA2(nn.Module):
    def __init__(self):
        super(NetworkA2, self).__init__()
        # [50, 60, 100, 0, .... , 20, 8]
        self.fc_layer1 = nn.Linear(11, 15) ##CHANGE!! 6
        self.fc_layer2 = nn.Linear(15, 20)
        self.fc_layer3 = nn.Linear(20, 3) ##CHANGE!! 4
    def forward(self, x):
        # x is input
        # Layer 1
        x = F.relu(self.fc_layer1(x))
        # Layer 2
        x = F.relu(self.fc_layer2(x))
        # Layer 3
        x = self.fc_layer3(x)
        return x
model2 = NetworkA2()

In [7]:
#reset the environment
def reset():
    sumoBinary = checkBinary('sumo') #-gui
    traci.start([sumoBinary, "-c", "map_grid_lock_add2TL_copy.sumocfg",
                             "--tripinfo-output", "tripinfo.xml", '--start','true','--quit-on-end','true','--time-to-teleport','-1'])

In [8]:
id = {'virtual' : '270329335' ,'KasemRat': 'cluster_272448137_272555800_272555808_7660045934_7710268409'}
detecters = {'QA' : ['156261350#0'], 'QB' : ['-453962283#3'], 'QC' : ['459551209#0', '459551209#3'], 'QD' : ['27702347#0', '27702347#4', '27702347#6']}

In [9]:
import sumolib
import lxml.etree as ET
def get_edge_id(POI):
    root = ET.Element('additionals')
    net = sumolib.net.readNet('map_grid_lock_add2TL_copy_newTLS.net.xml')
    output = []
    for key in POI:
        edgeID = net.getEdge(key)
        numOfLane = edgeID.getLaneNumber()
        for lane_index in range(numOfLane):
            output.append(key+'_'+str(lane_index))
    return output    

In [10]:
def get_state():
    occupancy = []
    for key in ['QA', 'QB', 'QC', 'QD']:
        occupancy.append(sum([traci.lanearea.getLastStepOccupancy(e) for e in get_edge_id(detecters[key])])/len(get_edge_id(detecters[key])))
    current_phase = [traci.trafficlight.getPhase(id[key]) for key in ['virtual','KasemRat']]
    for i in current_phase:
        current_phase_binary = [0, 0, 0]
        current_phase_binary[int(i//2)] = 1
        occupancy+= current_phase_binary
    return occupancy

In [11]:
reward_memory = []
def plot_durations():
    print('show')
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(reward_memory, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated

In [12]:
def get_action_junction(current_state):
    if np.random.rand() < epsilon:
        action = np.random.randint(1)
    else:
        # Ask model what to do?        
        answer = policy_net1(torch.tensor(current_state))
        action = torch.argmax(answer)
    return action

In [15]:
def take_action(action, ID):
    w = 0.999
    act = [5, 1, 3]
    print(act[action])
    change_to_phase = (act[action]+1)%6
    if change_to_phase == traci.trafficlight.setPhase(id[junction_name[ID]]):
        traci.trafficlight.setPhase(id[junction_name[ID]], change_to_phase) # NO YELLOW PHASE 
    else:
        traci.trafficlight.setPhase(id[junction_name[ID]], act[action]) # YELLOW PHASE FIRST
    for i in range(5):
        traci.simulationStep()
#     w1 = sum([traci.lanearea.getLastStepOccupancy(e) for e in get_edge_id(reward_w1)])
#     w2 = sum([traci.lanearea.getLastStepOccupancy(e) for e in get_edge_id(reward_w1)])
#     reward = -w2*(1-w) - w1*(w)
    reward = -sum(get_state())
    print('reward', reward)
    next_state1 = get_state()
    junct = get_action_junction( next_state1)
    next_state2 = copy.deepcopy( next_state1)
    print('current_state2', next_state2)
    next_state2.append(int(junct))
    return reward, next_state1, next_state2

In [16]:
def get_action_phase(current_state):
    if np.random.rand() < epsilon:
        action = np.random.randint(2)
    else:
        # Ask model what to do?        
        answer = policy_net2(torch.tensor(current_state))
        action = torch.argmax(answer)
    return action

In [17]:
def initial(state1, state2, max_occupancy):
    occupancy = copy.deepcopy(state1[0:4])
    print(occupancy)
    s1 = copy.deepcopy(state1[4:7])
    s2 = copy.deepcopy(state1[7:9])
    if state1 == None and state2 == None:
        if ((occupancy[0] >= max_occupancy) or (occupancy[3] >= max_occupancy)) and s1[1] != 1:
            print(occupancy)
            print('set0')
            state2 = copy.deepcopy(state1)
            state2.append(0)
            traci.trafficlight.setPhase(id['virtual'], 2)
            junct = 0
            action = 1
        elif ((occupancy[0] >= max_occupancy) or (occupancy[3] >= max_occupancy)) and s2[1] != 1:
            traci.trafficlight.setPhase(id['KasemRat'], 2)
            state2 = copy.deepcopy(state1)
            state2.append(1)
            junct = 1
            action = 1
        elif occupancy[2] >= max_occupancy:
            print(occupancy)
            print('set2')
            traci.trafficlight.setPhase(id['KasemRat'], 2)
            state2 = copy.deepcopy(state1)
            state2.append(1)
            junct = 1
            action = 1
        else:
            traci.trafficlight.setPhase(id['virtual'], 0)
            state2 = copy.deepcopy(state1)
            state2.append(0)
            junct = 1
            action = 0
    else:
        if ((occupancy[0] >= max_occupancy) or (occupancy[3] >= max_occupancy)) and s1[1] != 1:
            print(occupancy)
            print('set0')
            traci.trafficlight.setPhase(id['virtual'], 2)
            junct = 0
            action = 1
        elif ((occupancy[0] >= max_occupancy) or (occupancy[3] >= max_occupancy)) and s2[1] != 1:
            traci.trafficlight.setPhase(id['KasemRat'], 2)
            junct = 1
            action = 1
        elif occupancy[2] >= max_occupancy:
            print(occupancy)
            print('set2')
            traci.trafficlight.setPhase(id['KasemRat'], 2)
            junct = 1
            action = 1
        else:
            traci.trafficlight.setPhase(id['virtual'], 0)
            junct = 1
            action = 0
    for i in range(5):
        traci.simulationStep()
    reward = -sum(get_state())
    next_state1 = get_state()
    next_state2 = copy.deepcopy(next_state1)
    if ((occupancy[0] >= max_occupancy) or (occupancy[3] >= max_occupancy)) and s1[1] != 1:
        next_state2.append(0)
    elif ((occupancy[0] >= max_occupancy) or (occupancy[3] >= max_occupancy)) and s2[1] != 1:
        next_state2.append(1)
    elif occupancy[2] >= max_occupancy:
        next_state2.append(1)
    else:
        next_state2.append(0)
    
    return reward, state1, state2, action, junct, next_state1, next_state2

In [18]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [19]:
def optimize_model1():
    if len(memory1) < BATCH_SIZE:
        return

    transitions = memory1.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), dtype=torch.bool)
#     print('non_final_mask', non_final_mask)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
#     print('non_final_next_states',  non_final_next_states)
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
#     print('state_batch', state_batch)
#     print('reward_batch', reward_batch)
#     print('action_batch', action_batch)
    state_action_values = policy_net1(state_batch).gather(1, action_batch)
#     print(state_action_values)
    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE)
    next_state_values[non_final_mask] = target_net1(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
#     print('state_action_values', state_action_values)
#     print('expected_state_action_values ', expected_state_action_values )

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer1.zero_grad()
    loss.backward()
    for param in policy_net1.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer1.step()

In [20]:
# non_final_mask = torch.tensor([True, True, True, True, True, True, True, True, True, True])
# non_final_next_states = torch.tensor([[ 7.1332,  6.4395, 22.7967,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,
#           0.0000,  0.0000,  0.0000],
#         [ 9.9864, 16.0987, 18.4670,  4.4428,  0.0000,  0.0000,  1.0000,  1.0000,
#           0.0000,  0.0000,  0.0000],
#         [ 9.9864, 14.0199, 18.6225,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,
#           0.0000,  0.0000,  0.0000],
#         [ 9.9864, 16.0987, 16.0655, 23.2155,  1.0000,  0.0000,  0.0000,  0.0000,
#           1.0000,  0.0000,  1.0000],
#         [ 9.9864, 15.5202, 20.6725, 27.4631,  1.0000,  0.0000,  0.0000,  0.0000,
#           1.0000,  0.0000,  1.0000],
#         [ 9.9864, 16.2855, 21.2403, 34.1072,  1.0000,  0.0000,  0.0000,  0.0000,
#           1.0000,  0.0000,  1.0000],
#         [ 1.8350,  0.0000, 17.5268,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,
#           0.0000,  0.0000,  0.0000],
#         [ 9.9864, 16.0987, 21.6002, 11.9312,  0.0000,  0.0000,  1.0000,  0.0000,
#           1.0000,  0.0000,  0.0000],
#         [ 0.0000,  0.0000, 16.1663,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,
#           0.0000,  0.0000,  0.0000],
#         [ 9.9864, 16.0987, 14.5397, 37.0644,  1.0000,  0.0000,  0.0000,  0.0000,
#           1.0000,  0.0000,  1.0000]])

# state_batch = torch.tensor([[ 1.8350,  0.0000, 17.5268,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,
#           0.0000,  0.0000,  0.0000],
#         [ 9.9864, 14.0199, 18.6225,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,
#           0.0000,  0.0000,  0.0000],
#         [ 7.1332,  6.4395, 22.7967,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,
#           0.0000,  0.0000,  0.0000],
#         [ 9.9864, 16.0987, 21.6002, 11.9312,  0.0000,  0.0000,  1.0000,  0.0000,
#           1.0000,  0.0000,  0.0000],
#         [ 9.9864, 16.0987, 16.0655, 23.2155,  1.0000,  0.0000,  0.0000,  0.0000,
#           1.0000,  0.0000,  1.0000],
#         [ 9.9864, 15.5202, 20.6725, 27.4631,  1.0000,  0.0000,  0.0000,  0.0000,
#           1.0000,  0.0000,  1.0000],
#         [ 0.0000,  0.0000, 16.1663,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,
#           0.0000,  0.0000,  0.0000],
#         [ 9.9864, 16.0987, 18.4670,  4.4428,  0.0000,  0.0000,  1.0000,  1.0000,
#           0.0000,  0.0000,  0.0000],
#         [ 0.0000,  0.0000,  8.8183,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,
#           0.0000,  0.0000,  0.0000],
#         [ 9.9864, 16.2855, 21.2403, 34.1072,  1.0000,  0.0000,  0.0000,  0.0000,
#           1.0000,  0.0000,  1.0000]])
# action_batch = torch.tensor([[0],
#         [2],
#         [0],
#         [0],
#         [0],
#         [0],
#         [0],
#         [2],
#         [0],
#         [0]])
# reward_batch = torch.tensor([[0],
#         [2],
#         [0],
#         [0],
#         [0],
#         [0],
#         [0],
#         [2],
#         [0],
#         [0]])
# state_action_values = torch.tensor([[ 0.2508],
#         [-0.0413],
#         [ 0.1092],
#         [-0.0687],
#         [-0.0691],
#         [-0.0643],
#         [ 0.2510],
#         [-0.0855],
#         [ 0.2228],
#         [-0.0836]])
# state_action_values = torch.tensor([[ 0.2508],
#         [-0.0413],
#         [ 0.1092],
#         [-0.0687],
#         [-0.0691],
#         [-0.0643],
#         [ 0.2510],
#         [-0.0855],
#         [ 0.2228],
#         [-0.0836]])
# expected_state_action_values = torch.tensor([[-38.2710, -38.4463, -38.4064, -38.4315, -38.4271, -38.4445, -38.1435,
#          -38.4311, -38.1434, -38.4599],
#         [-50.8967, -51.0719, -51.0321, -51.0571, -51.0528, -51.0702, -50.7692,
#          -51.0568, -50.7691, -51.0856],
#         [-44.5306, -44.7058, -44.6659, -44.6910, -44.6867, -44.7040, -44.4031,
#          -44.6907, -44.4029, -44.7194],
#         [-67.2678, -67.4431, -67.4032, -67.4283, -67.4240, -67.4413, -67.1403,
#          -67.4279, -67.1402, -67.4567],
#         [-75.5440, -75.7192, -75.6794, -75.7044, -75.7001, -75.7175, -75.4165,
#          -75.7041, -75.4163, -75.7328],
#         [-83.5211, -83.6964, -83.6565, -83.6816, -83.6773, -83.6946, -83.3937,
#          -83.6813, -83.3935, -83.7100],
#         [-21.2635, -21.4388, -21.3989, -21.4240, -21.4197, -21.4370, -21.1361,
#          -21.4237, -21.1359, -21.4524],
#         [-61.5182, -61.6934, -61.6536, -61.6786, -61.6743, -61.6917, -61.3907,
#          -61.6783, -61.3905, -61.7070],
#         [-18.0681, -18.2433, -18.2035, -18.2285, -18.2242, -18.2416, -17.9406,
#          -18.2282, -17.9404, -18.2570],
#         [-79.5910, -79.7663, -79.7264, -79.7515, -79.7471, -79.7645, -79.4635,
#          -79.7511, -79.4634, -79.7799]])
# state_action_values = policy_net2(state_batch).gather(1, action_batch)
# next_state_values = torch.zeros(BATCH_SIZE)
# print(next_state_values)
# next_state_values[non_final_mask] = target_net2(non_final_next_states).max(1)[0].detach()
# print(next_state_values[non_final_mask])
# print(state_action_values)
#     # Compute the expected Q values
# expected_state_action_values = (next_state_values * GAMMA) + reward_batch
# print(expected_state_action_values)

In [21]:
def optimize_model2():
    if len(memory2) < BATCH_SIZE:
        return
#     print(len(memory2))
    transitions = memory2.sample(BATCH_SIZE)
#     print(type(zip(*transitions)))
#     print(type(Transition(*zip(*transitions))))
    batch = Transition(*zip(*transitions))
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), dtype=torch.bool)
#     print('non_final_mask', non_final_mask)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
#     print('non_final_next_states',  non_final_next_states)
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
#     print('state_batch', state_batch)
#     print('reward_batch', reward_batch)
#     print('action_batch', action_batch)
    state_action_values = policy_net2(state_batch).gather(1, action_batch)
#     print(state_action_values)
    next_state_values = torch.zeros(BATCH_SIZE)
    next_state_values[non_final_mask] = target_net2(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
#     print('state_action_values', state_action_values)
#     print('expected_state_action_values ', expected_state_action_values )
    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
    # Optimize the model
    optimizer2.zero_grad()
    loss.backward()
    for param in policy_net2.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer2.step()

In [22]:
junction_name = ['virtual','KasemRat']

time_step = 5
reward = 0 
epsilon = 0.1 # ดมกาว 10%
GAMMA = 0.9  #GAMMA = [e for e in gamma()] ภีมเปลี่ยนเอาเองนะถ้าอยากเปลี่ยน

action = 0
current_state1 = None
previous_state1 = None

memory = []
BATCH_SIZE = 16

start_training_step = 10
current_training_step = 0 

alpha = 0.1

# create your optimizer
optimizer1 = optim.SGD(model1.parameters(), lr=0.01)
optimizer2 = optim.SGD(model2.parameters(), lr=0.01)
criterion = nn.MSELoss()

policy_net1 = model1
target_net1 = model1
target_net1.load_state_dict(policy_net1.state_dict())
target_net1.eval()
optimizer1 = optim.RMSprop(policy_net1.parameters())

policy_net2 = model2
target_net2 = model2
target_net2.load_state_dict(policy_net2.state_dict())
target_net2.eval()
optimizer2 = optim.RMSprop(policy_net2.parameters())

memory1 = ReplayMemory(10000)
memory2 = ReplayMemory(10000)
TARGET_UPDATE = 10

steps_done = 0

In [23]:
traci.close()

FatalTraCIError: Not connected.

In [24]:
#episode = 0
state1 = None
state2 = None
rewards = 0
reset()
traci.simulationStep()
state1 = get_state()
for seconds in range(100):
    reward, current_state1, current_state2, action, junct, next_state1, next_state2 = initial(state1, state2, 50)
    if state1 is not None and state2 is not None:
        memory1.push(torch.tensor([current_state1]), torch.tensor([[junct]]), torch.tensor([next_state1]), torch.tensor([reward]))
        memory2.push(torch.tensor([current_state2]), torch.tensor([[action]]), torch.tensor([next_state2]), torch.tensor([reward]))
    state1 = next_state1
    state2 = next_state2
    optimize_model1()
    optimize_model2()
    rewards += reward
reward_memory.append(rewards)
traci.close()

[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 8.81831169220976, 0.0]
[0.0, 0.0, 16.067782894979423, 0.0]
[0.0, 0.0, 16.166341431316717, 0.0]
[0.0, 0.0, 17.899592152070632, 0.0]
[1.8350440624129567, 0.0, 17.52676374436763, 0.0]
[4.279891304347826, 0.0, 21.235858225237195, 0.0]
[7.133152173913045, 6.439473134016308, 22.796659128910978, 0.0]
[8.559782608695654, 11.409742513476138, 19.994779610745667, 0.0]
[9.986413043478262, 14.019908683823, 18.622492776693996, 0.0]
[9.986413043478262, 16.09868283504077, 15.397939185622702, 2.221420843851424]
[9.986413043478262, 16.09868283504077, 18.467009217365618, 4.442841687702848]
[9.986413043478262, 16.09868283504077, 21.93184201676884, 5.629493407041392]
[9.986413043478262, 16.09868283504077, 21.60015148394345, 11.931180900189112]
[9.986413043478262, 16.09868283504077, 18.457990401523453, 17.8968228386569]
[9.986413043478262, 16.09868283504077, 16.065528299267825, 23.215467077912734]
[9.986413043478262, 16.09868283504077, 22.47150963582653, 22.61937378798184]
[9

In [25]:
# async def run():
for episode in range(100):
    count = 0
    reset()
    rewards = 0
    traci.simulationStep()
    previous_phase = [traci.trafficlight.getPhase(id[key]) for key in ['virtual','KasemRat']]
#     print(previous_phase)
    for seconds in range(100): 
#         traci.simulationStep()
#             if seconds >= 300: ??
#         if seconds%time_step == 0:
#             if done(count) >= 50:
#                 print('done')
#                 break
        if current_state1 is None:
            current_phase = [traci.trafficlight.getPhase(id[key]) for key in ['virtual','KasemRat']]
#             print('current_phase', current_phase)
            current_state1 = get_state()
#             print('current_state1', current_state1)
            junct = get_action_junction(current_state1)
            current_state2 = copy.deepcopy(current_state1)
#             print('current_state2', current_state2)
            current_state2.append(int(junct))
#             print('current_state2', current_state2)
        else:
            current_state1 = next_state1
            current_state2 = next_state2
        action = get_action_phase(current_state2)
#         print('junction', junct)
#         print('action', action)
        reward, next_state1, next_state2 = take_action(action, junct) 
        if previous_state1 is not None:
            memory1.push(torch.tensor([current_state1]), torch.tensor([[junct]]), torch.tensor([next_state1]), torch.tensor([reward]))
            memory2.push(torch.tensor([current_state2]), torch.tensor([[action]]), torch.tensor([next_state2]), torch.tensor([reward]))
        optimize_model1()
        optimize_model2()
        previous_state1 = current_state1
        previous_state2 = current_state2
        rewards += reward
    reward_memory.append(rewards)
    plot_durations()

    if episode % TARGET_UPDATE == 0:
        target_net1.load_state_dict(policy_net1.state_dict())
        target_net2.load_state_dict(policy_net2.state_dict())
    traci.close()

print('Complete')
plt.ioff()
plt.show()

3


TypeError: setPhase() missing 1 required positional argument: 'index'